In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
import keras

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow.keras import regularizers
import tensorflow as tf

In [2]:
X_train = load('trainX.npy')
X_test = load('testX.npy')
y_train = load('trainy.npy')
y_test = load('testy.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [3]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [4]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

In [5]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=10, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=10, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 119, 64)           3904      
                                                                 
 conv1d_1 (Conv1D)           (None, 110, 64)           41024     
                                                                 
 dropout (Dropout)           (None, 110, 64)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 55, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3520)              0         
                                                                 
 dense (Dense)               (None, 128)               450688    
                                                        

In [6]:
history = model.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=128, callbacks=callbacks)

Epoch 1/200
207/207 [==============================] - 26s 111ms/step - loss: 3.5671 - accuracy: 0.2868 - val_loss: 2.1038 - val_accuracy: 0.6315
Epoch 2/200
207/207 [==============================] - 24s 115ms/step - loss: 1.3071 - accuracy: 0.7710 - val_loss: 0.9383 - val_accuracy: 0.8301
Epoch 3/200
207/207 [==============================] - 24s 116ms/step - loss: 0.6925 - accuracy: 0.8764 - val_loss: 0.6125 - val_accuracy: 0.8857
Epoch 4/200
207/207 [==============================] - 25s 121ms/step - loss: 0.4706 - accuracy: 0.9125 - val_loss: 0.4482 - val_accuracy: 0.9147
Epoch 5/200
207/207 [==============================] - 25s 121ms/step - loss: 0.3535 - accuracy: 0.9336 - val_loss: 0.3608 - val_accuracy: 0.9328
Epoch 6/200
207/207 [==============================] - 24s 114ms/step - loss: 0.2830 - accuracy: 0.9450 - val_loss: 0.3108 - val_accuracy: 0.9388
Epoch 7/200
207/207 [==============================] - 25s 122ms/step - loss: 0.2347 - accuracy: 0.9535 - val_loss: 0.2798 -

In [7]:
loss, accuracy = model.evaluate(X_validation, y_validation, verbose=True)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=True)
print("Testing Accuracy: {:.4f}".format(accuracy))

207/207 [==============================] - 1s 6ms/step - loss: 0.1530 - accuracy: 0.9607
Validation Accuracy:  0.9607
117/117 [==============================] - 1s 6ms/step - loss: 0.7096 - accuracy: 0.8914
Testing Accuracy: 0.8914
